In [ ]:
!pip install python-dotenv

In [ ]:
!pip install openai

In [ ]:
import openai
import getpass

import os

from dotenv import load_dotenv
from openai import OpenAI

# Load .env file
load_dotenv(".env")

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

Enter API key for OpenAI: ··········


In [ ]:
def offer_chain(reason):
     # Dynamic offer based on reason
        offer_prompt = f"""
          The user has stated their reason for canceling is: {reason}.
          Please generate a fun, space-themed response with an appropriate retention offer.
          Make sure to include the following:
          - Address the reason empathetically.
          - Suggest only one of the available offers, whichever is most likely to retain the customer: 50% discount, pause for 6 months, or skip next order.
          - Keep the tone light and fun.
          - Keep it concise
          """
        try:
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a fun, helpful, space-themed chatbot that helps customer retention by offering best suitable retention offers as given below."},
                    {"role": "user", "content": offer_prompt},
                ],
                temperature=0.7,  #I tried varying this value from 0.7-1.1, 0.7 gave the best results
            )

            offer_response = response.choices[0].message.content
            return offer_response
        except openai.error.AuthenticationError:
            return "Authentication Error: Check your API key."
        except Exception as e:
            return f"An unexpected error occurred: {e}"

# performing sentiment analysis to detect negative sentiment (could also do emotional analysis to get to more granularity)
def analyze_sentiment(text):
    prompt = f"Analyze the sentiment of the following text and respond with only one word - 'positive', 'negative', or 'neutral': {text}"
    response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful chatbot that analyzes the sentiment"},
                    {"role": "user", "content": prompt},
                ],
            )
    sentiment = response.choices[0].message.content
    return sentiment.strip().lower()

def prompt_chain():
    # Step 1: Reason Identification
    reasons_prompt = """
      🚀 Greetings, Captain! I see you’ve activated the ‘Cancel My Subscription’ sequence.
      Before we finalize your request, can you share why you’re choosing to beam out of our coffee subscription galaxy?

      Here are a few common reasons to help narrow it down:
      1️⃣ Too expensive
      2️⃣ Too much coffee stocked up
      3️⃣ Feeling too jittery
      4️⃣ Not jittery or energy boost enough
      5️⃣ Not a fan of the taste
      6️⃣ Moving to a new house

      Please type the number corresponding to your reason."""

    print(reasons_prompt)
    reason = input("Your reason (1-6): ").strip()

    # Step 2: Generate Retention Offer
    if reason in ["1", "2", "3", "4", "5", "6"]:
        reason_mapping = {
            "1": "too expensive",
            "2": "too much coffee stocked up",
            "3": "feeling too jittery",
            "4": "not enough jittery or energy boost",
            "5": "not a fan of the taste",
            "6": "moving to a new house",
        }
        # contitional statements for reasons 3,4,5 to suggest the subscribers to try new blends
        if reason in ["3", "4", "5"]:


            print(""" Please select the Bean Me Up blend that you are currently using from the list below:

                  1️⃣ Decaf: Live Long and De-Caffeinate with 10 mg of caffeine and hints of hazelnut
                  2️⃣ Half-Caf: One Small Sip for Man with 50 mg of caffeine and hints of vanilla
                  3️⃣ Regular Caf: Where No Bean Has Been Before with 100 mg of caffeine
                  4️⃣ Super Caf: Warp Speed Blend with 200 mg of caffeine
                  """)
          # Gather the current product data from the user
            current_blend = input("Your blend (1-4): ").strip()

            blend_mapping = {
                "1": "Decaf: Live Long and De-Caffeinate with 10 mg of caffeine and hints of hazelnut",
                "2": "Half-Caf: One Small Sip for Man with 50 mg of caffeine and hints of vanilla",
                "3": "Regular Caf: Where No Bean Has Been Before with 100 mg of caffeine",
                "4": "Super Caf: Warp Speed Blend with 200 mg of caffeine",
            }

            # handling the edge cases of the customer already is subscribed to the perfect blend
            if(reason=="4" and current_blend=="4"):
                print("\nYou're already cruising with the highest caffeine blend we have—200 mg is our pinnacle of cosmic energy!, it's sad that the starship hasn't discovered more a stronger blend")
                print("\n We would like to offer you a special discount to give you little more time to adjust and absorb that cosmic energy")

                objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()

                if objection == "yes":
                    #call the offer prompt function
                    offer_response= offer_chain(reason_mapping[reason])
                    print(offer_response)

                    objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()

                    if objection == "yes":
                        print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                        return
                    elif objection=="no":
                        print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                        print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! ")
                        return
                    else:
                         print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                         prompt_chain()


                elif objection=="no":
                    print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                    print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead!")
                    return

                else:
                  # checking the sentiment score of the user input response
                        sentiment= analyze_sentiment(objection)

                        if(sentiment=='negative'):
                          prompt=""" We’re sorry to hear you’re feeling this way, Captain. we’d love to make things right.
                                   How about trying a new blend, skipping your next delivery, or pausing your subscription for a while?"""
                          print(prompt)
                          objection_new= input("/nDoes these options work for you? (yes/no)").strip().lower()

                          if objection_new =='yes':
                            offer_response= offer_chain(reason_mapping[reason])
                            print(offer_response)
                            objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()
                            if objection == "yes":
                                print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                                return
                            elif objection=="no":
                                print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                return
                            else:
                                print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                                prompt_chain()

            elif(reason=="3" and current_blend=="1"):
                print("\nYou're already at lowest caffeine blend we have—10 mg is our lowest of cosmic energy!, it's sad that the starship hasn't discovered more a lighter blend")
                print("\n We would like to offer you a special discount to give you little more time to adjust and absorb that cosmic energy")

                objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()

                if objection == "yes":
                    #call the offer prompt function
                    offer_response= offer_chain(reason_mapping[reason])
                    print(offer_response)

                    objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()
                    if objection == "yes":
                        print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                        return
                    elif objection=="no":
                        print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                        print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                        return
                    else:
                        print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                        prompt_chain()
                elif(objection=="no"):
                    print("\nUnderstood, Captain. Your subscription has been successfully canceled. We're sorry to see you go!")
                    print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead!")
                    return
                else:

                  # checking the sentiment score of the user input response
                        sentiment= analyze_sentiment(objection)

                        if(sentiment=='negative'):
                          prompt=""" We’re sorry to hear you’re feeling this way, Captain. we’d love to make things right.
                                   How about trying a new blend, skipping your next delivery, or pausing your subscription for a while?"""
                          print(prompt)
                          objection_new= input("/nDoes these options work for you? (yes/no)").strip().lower()

                          if objection_new =='yes':
                            offer_response= offer_chain(reason_mapping[reason])
                            print(offer_response)
                            objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()
                            if objection == "yes":
                                print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                                return
                            elif objection=="no":
                                print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                return
                            else:
                                print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                                prompt_chain()
                        else:
                                 print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                 print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                 return

            else:
                offer_prompt = f"""
                            The user has stated the reason for canceling is {reason_mapping[reason]}. Suggest them
                            to try out our other products based on the reason stated given the current product that they use is {blend_mapping[current_blend]}.

                            Available products are:
                            - Decaf: Live Long and De-Caffeinate with 10 mg of caffeine and hints of hazelnut
                            - Half-Caf: One Small Sip for Man with 50 mg of caffeine and hints of vanilla
                            - Regular Caf: Where No Bean Has Been Before with 100 mg of caffeine
                            - Super Caf: Warp Speed Blend with 200 mg of caffeine

                            If the reason stated is them feeling too jittery, then suggest the blends that have less caffeine. Whereas, if they are
                            feeling not enough jittery or not getting enough energy, suggest the products that are high in caffeine.
                            If the reason is that they are not a fan of the taste, suggest the blends that have hints of hazelnut and vanilla.

                            In the case that they already are on the perfect blend, then don't suggest any other product. Instead, in a space-themed funny way, respond concisely that they are amazing and that they already are on the perfect blend.

                            Make sure to include the following:
                                - Address the reason empathetically.
                                - Keep the tone light and fun.
                    """

                response = client.chat.completions.create(
                        model="gpt-4o",
                        messages=[
                            {"role": "system", "content": "You are a fun, space-themed chatbot."},
                            {"role": "user", "content": offer_prompt},
                        ],
                        temperature=0.7,
                    )

                offer_response = response.choices[0].message.content
                print(offer_response)

                objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()

                if objection == "yes":
                    print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                    return
                elif objection =="no":
                    print("\nWe would like to offer you a special discount to give us a re-entry")
                    offer_response= offer_chain(reason_mapping[reason])
                    print(offer_response)

                    objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()

                    if objection == "yes":
                        print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                        return
                    elif objection=="no":
                        print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                        print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                        return
                    else:
                        print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again?")
                        prompt_chain()
                else:
                  # checking the sentiment score of the user input response
                        sentiment= analyze_sentiment(objection)

                        if(sentiment=='negative'):
                          prompt=""" We’re sorry to hear you’re feeling this way, Captain. we’d love to make things right.
                                   How about trying a new blend, skipping your next delivery, or pausing your subscription for a while?"""
                          print(prompt)
                          objection_new= input("/nDoes these options work for you? (yes/no)").strip().lower()

                          if objection_new =='yes':
                            offer_response= offer_chain(reason_mapping[reason])
                            print(offer_response)
                            objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()
                            if objection == "yes":
                                print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                                return

                            elif objection=="no":
                                              objection_prompt = f"""
                                  The user declined the first retention offer of {offer_response}. Suggest an alternative (e.g., skip next order) in a fun, space-themed tone.
                                  Keep it short and engaging.
                                  """

                                              objection_response = client.chat.completions.create(
                                      model="gpt-4o",
                                      messages=[
                                          {"role": "system", "content": "You are a fun, space-themed chatbot."},
                                          {"role": "user", "content": objection_prompt},
                                      ],
                                      temperature=0.7,
                                  )

                                              print(objection_response.choices[0].message.content)
                                              final_decision = input("\nDoes this work for you now? (yes/no): ").strip().lower()
                                              if(final_decision=="no"):

                                                print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                                print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                                return
                                              else:
                                                print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                            else:
                                print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                                prompt_chain()
                          else:
                                 print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                 print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                 return
                        else:
                           print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                           prompt_chain()




        #call offer chain
        offer_response=offer_chain(reason_mapping[reason])
        print(offer_response)

        # Step 3: Handle Objections
        objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()
        if objection == "no":
            objection_prompt = f"""
                  The user declined the first retention offer of {offer_response}. Suggest an alternative (e.g., skip next order) in a fun, space-themed tone.
                  Keep it short and engaging.
                  """

            objection_response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a fun, space-themed chatbot."},
                    {"role": "user", "content": objection_prompt},
                ],
                temperature=0.7,
            )

            print(objection_response.choices[0].message.content)
            final_decision = input("\nDoes this work for you now? (yes/no): ").strip().lower()

            if final_decision == "no":
                print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! ")
            elif final_decision=="yes":
                print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
            else:
              # checking the sentiment score of the user input response
                        sentiment= analyze_sentiment(final_decision)

                        if(sentiment=='negative'):
                          prompt=f"""\nWe’re sorry to hear you’re feeling this way, Captain. we’d love to make things right.
                                     How about trying a new blend, skipping your next delivery, or pausing your subscription for a while?"""

                          print(prompt)
                          objection_new= input("\nDoes these options work for you? (yes/no)").strip().lower()

                          if objection_new =='yes':
                            offer_response= offer_chain(reason_mapping[reason])
                            print(offer_response)
                            objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()
                            if objection == "yes":
                                print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                                return
                            elif objection=="no":
                                print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                return
                            else:
                                print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                                prompt_chain()
                          else:
                                 print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                 print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                 return
        elif objection=="yes":
            print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")

        else:
          # checking the sentiment score of the user input response
                        sentiment= analyze_sentiment(objection)

                        if(sentiment=='negative'):
                          prompt=f"""\nWe’re sorry to hear you’re feeling this way, Captain. we’d love to make things right.
                                     How about trying a new blend, skipping your next delivery, or pausing your subscription for a while?"""

                          print(prompt)
                          objection_new= input("\nDoes these options work for you? (yes/no)").strip().lower()

                          if objection_new =='yes':
                            offer_response= offer_chain(reason_mapping[reason])
                            print(offer_response)
                            objection = input("\nDoes this option work for you? (yes/no): ").strip().lower()
                            if objection == "yes":
                                print("\nFantastic! Your new course is set. Live long and caffeinate! ☕️")
                                return
                            elif objection=="no":
                                print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                return
                            else:
                                print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
                                prompt_chain()
                          else:
                                 print("\nUnderstood, Captain. Your subscription has been successfully canceled.")
                                 print("If you ever wish to rejoin our galaxy of great coffee, we’ll be here. Warp speed ahead! 🌟")
                                 return
    else:
        print("\n🤔 Hmm, that’s not a choice in our galaxy. Could you try again with one of the listed reasons?")
        prompt_chain()

# Run the prompt chain
prompt_chain()




      🚀 Greetings, Captain! I see you’ve activated the ‘Cancel My Subscription’ sequence.
      Before we finalize your request, can you share why you’re choosing to beam out of our coffee subscription galaxy?

      Here are a few common reasons to help narrow it down:
      1️⃣ Too expensive
      2️⃣ Too much coffee stocked up
      3️⃣ Feeling too jittery
      4️⃣ Not jittery or energy boost enough
      5️⃣ Not a fan of the taste
      6️⃣ Moving to a new house

      Please type the number corresponding to your reason.
Your reason (1-6): 2
🚀 Greetings, caffeinated cosmic traveler! 🌌

We totally understand—sometimes it feels like your coffee stash could fuel a rocket to the moon! 🌕 No worries, we’ve got a stellar solution: how about we skip your next order and give your caffeine nebula a chance to settle? 🌟

This way, you can orbit back into action when your coffee reserves are ready for a boost! Let us know if you'd like to engage warp speed on this offer. 🚀☕✨

Safe travels thr

KeyboardInterrupt: Interrupted by user